### Ingestion del archivo "language_role.json"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
language_role_schema = StructType(
    fields=[
        StructField("roleId", IntegerType(), True),
        StructField("languageRole", StringType(), True)
    ]
)

In [0]:
language_role_df = spark.read \
    .schema(language_role_schema) \
    .option("multiline", True) \
    .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas

1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
language_role_final_df = add_ingestion_date(language_role_df) \
    .withColumnsRenamed({
        "roleId": "role_id",
        "languageRole": "language_role",
    }) \
    .withColumns({
        "environment": lit(v_environment)
    }) \
    .withColumn("file_date", lit(v_file_date))

#### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
language_role_final_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("movie_silver.languages_roles")

In [0]:
dbutils.notebook.exit("Success")